In [1]:
#import libraries & modules 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import style
style.use('ggplot')
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#importing necessary packages 
import pandas as pd
from pandas.plotting import autocorrelation_plot

import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

#importing statsmodel 
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


#importing scikit learn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as MSE, make_scorer, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

#miscellaneous libraries
from datetime import datetime
import warnings
from math import sqrt
from collections import Counter

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
plt.style.use('ggplot')
%matplotlib inline

## Zillow Data

### Median Sale Price

In [2]:
#Median Sale Price: The median price at which homes across various geographies were sold.
#https://www.zillow.com/research/data/
median_price = pd.read_csv('data/Metro_median_sale_price_uc_sfrcondo_month.csv')
median_price.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2008-02-29,2008-03-31,2008-04-30,2008-05-31,2008-06-30,...,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28
0,102001,0,United States,country,NaN,180000.0,185000.0,185000.0,187000.0,193000.0,...,365000.0,365000.0,353143.0,346000.0,340000.0,335000.0,328000.0,323000.0,319184.0,329900.0
1,394913,1,"New York, NY",msa,NY,404586.0,399000.0,399001.0,400000.0,405000.0,...,539995.0,560000.0,575000.0,575000.0,550000.0,530000.0,525000.0,515000.0,520000.0,515000.0
2,753899,2,"Los Angeles, CA",msa,CA,505000.0,496500.0,500000.0,490000.0,483000.0,...,919500.0,900000.0,876000.0,868000.0,850000.0,844500.0,833000.0,810000.0,815000.0,825000.0
3,394463,3,"Chicago, IL",msa,IL,238500.0,237825.0,241000.0,240000.0,250000.0,...,305500.0,315000.0,310000.0,300000.0,285000.0,280000.0,279000.0,275000.0,269000.0,274000.0
4,394514,4,"Dallas, TX",msa,TX,137000.0,144000.0,143900.0,145874.0,154750.0,...,425000.0,425000.0,410000.0,400000.0,388000.0,385000.0,374900.0,363692.0,350163.0,374450.0


In [3]:
#checking for missing values
median_price.isna().sum()

RegionID       0
SizeRank       0
RegionName     0
RegionType     0
StateName      1
              ..
2022-10-31    25
2022-11-30    23
2022-12-31    22
2023-01-31    18
2023-02-28     2
Length: 186, dtype: int64

In [4]:
#for this dataset only homes in Florida are relevant 
df = median_price.loc[median_price['StateName'] == 'FL']

In [5]:
#dropping Region ID because its just a unique identifier
#dropping Region Type becaise it is not needed for this analysis 
#dropping Size Rank because it is not needed for this analysis 
#dropping State Name because all data is from Florida
df.drop(['RegionID', 'RegionType', 'SizeRank', 'StateName'], axis=1, inplace=True)


In [6]:
#for this dataset the only necessary years are 2016 to 2018
#dropping all other data columns 
for col in df.columns:
    if '2008' in col:
        del df[col]

In [7]:
df.head()

,RegionName,2009-01-31,2009-02-28,2009-03-31,2009-04-30,2009-05-31,2009-06-30,2009-07-31,2009-08-31,2009-09-30,...,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28
8,"Miami, FL",183000.0,185000.0,180000.0,165000.0,172000.0,185000.0,190000.0,180000.0,177500.0,...,423000.0,425000.0,421375.0,415000.0,410000.0,419250.0,410000.0,410000.0,410000.0,425000.0
18,"Tampa, FL",145000.0,135000.0,143000.0,140000.0,140000.0,145500.0,147500.0,137250.0,136000.0,...,375000.0,386600.0,381250.0,370000.0,360000.0,370000.0,355000.0,359000.0,342375.0,350000.0
22,"Orlando, FL",180000.0,166500.0,165000.0,150000.0,150000.0,155000.0,155000.0,150000.0,150000.0,...,385000.0,400000.0,393000.0,385000.0,380300.0,375000.0,370000.0,369500.0,359900.0,375000.0
39,"Jacksonville, FL",169450.0,178000.0,170000.0,161500.0,172000.0,180000.0,172000.0,172500.0,162000.0,...,360000.0,375000.0,374425.0,365000.0,349900.0,353500.0,345000.0,337000.0,335500.0,335000.0
72,"North Port, FL",165000.0,170000.0,170000.0,160950.0,170000.0,175000.0,170000.0,175000.0,162500.0,...,469000.0,477000.0,472500.0,449950.0,460000.0,483700.0,450000.0,420000.0,420000.0,440000.0


## NOAA Data 
Average wind speed (AWND)    
Direction of fastest 2-minute wind (WDF2)      
Direction of fastest 5-second wind (WDF5)       
Fastest 2-minute wind speed (WSF2)        
Fastest 5-second wind speed (WSF5)       
Peak gust time (PGTM)      
Weather types (WT**)    

### Hurricane Matthew (2016)

In [8]:
#Hurricane Matthew data 
#From NOAA 
matthew = pd.read_csv('data/Matthew.csv')
matthew.describe()

,LATITUDE,LONGITUDE,ELEVATION,AWND,PGTM,WDF2,WDF5,WDMV,WSF2,WSF5,...,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11
count,35444.000000,35444.000000,35056.000000,2825.000000,583.000000,2824.000000,2768.000000,149.000000,2824.000000,2768.000000,...,51.0,861.0,0.0,0.0,0.0,0.0,202.0,0.0,3.0,6.0
mean,28.304427,-82.068018,17.257648,7.240520,1296.775300,134.316572,132.810694,66.542282,17.592210,23.448988,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0
std,1.622852,1.643817,16.471771,3.888597,427.445597,98.254841,99.692938,59.758322,5.615843,7.785193,...,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0
min,24.550659,-87.467244,0.000000,0.890000,1.000000,10.000000,10.000000,0.000000,6.900000,8.900000,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0
25%,27.122200,-82.492256,4.300000,4.470000,1112.500000,60.000000,60.000000,14.900000,14.100000,18.100000,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0
50%,28.248440,-81.763550,11.000000,6.490000,1311.000000,100.000000,90.000000,60.300000,17.000000,21.900000,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0
75%,29.765200,-80.915000,27.100000,9.400000,1515.500000,190.000000,200.000000,95.100000,21.000000,27.100000,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0
max,30.953800,-80.034979,85.000000,33.330000,2359.000000,360.000000,360.000000,257.200000,55.000000,138.900000,...,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0


## Hurricane Irma (2017)

In [9]:
#Hurricane Irma data 
#From NOAA 
irma = pd.read_csv('data/Irma.csv')
irma.head()

,STATION,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,PGTM,WDF2,WDF5,WDMV,...,WT02,WT03,WT04,WT05,WT06,WT07,WT08,WT09,WT10,WT11
0,US1FLOK0029,30.4109,-86.6475,4.9,2017-08-01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1FLOK0029,30.4109,-86.6475,4.9,2017-08-02,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1FLOK0029,30.4109,-86.6475,4.9,2017-08-03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US1FLOK0029,30.4109,-86.6475,4.9,2017-08-04,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US1FLOK0029,30.4109,-86.6475,4.9,2017-08-05,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Hurricane Michael (2018)

In [10]:
#Hurricane Michael data 
#From NOAA 
michael = pd.read_csv('data/Michael.csv')
michael.describe()

,LATITUDE,LONGITUDE,ELEVATION,AWND,PGTM,WDF2,WDF5,WDMV,WSF2,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT10,WT11
count,20968.000000,20968.000000,20836.000000,1472.000000,288.000000,1442.000000,1387.000000,74.000000,1442.000000,1387.000000,385.0,34.0,106.0,0.0,0.0,0.0,71.0,0.0,1.0,0.0
mean,28.344763,-82.014772,16.678292,7.831549,1275.795139,134.188627,130.331651,68.374324,17.171775,22.907498,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
std,1.526815,1.603151,15.711413,3.698486,540.424078,93.906286,91.152028,47.258972,5.343040,6.864297,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
min,24.553380,-87.467244,0.000000,0.000000,0.000000,10.000000,10.000000,0.000000,6.000000,8.100000,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
25%,27.277935,-82.463644,4.600000,5.370000,1003.500000,70.000000,70.000000,31.100000,14.100000,18.100000,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
50%,28.382235,-81.755900,10.700000,7.380000,1310.000000,100.000000,100.000000,48.500000,16.100000,21.900000,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
75%,29.605565,-80.959000,25.900000,10.070000,1523.250000,180.000000,170.000000,94.775000,19.900000,25.900000,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN
max,30.964030,-80.034979,85.000000,29.750000,2357.000000,360.000000,360.000000,200.100000,63.100000,89.000000,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN


#### Bay County Data 
Bay County was the primary area struck by hurricane Michael

In [11]:
#Bay County Data from Hurricane Michael 
#Taken from NOAA

baycounty = pd.read_csv(r'data\Bay_County.csv')
baycounty.describe()

,LATITUDE,LONGITUDE,ELEVATION,AWND,PGTM,PGTM_ATTRIBUTES,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT08,WT08_ATTRIBUTES
count,66.000000,66.000000,66.000000,8.000000,0.0,0.0,8.000000,6.000000,8.000000,6.000000,3.0,1.0,1.0,0.0,0.0
mean,30.257186,-85.708757,13.027273,8.891250,NaN,NaN,163.750000,175.000000,17.037500,30.533333,1.0,1.0,1.0,NaN,NaN
std,0.090057,0.168744,11.741138,4.944721,NaN,NaN,96.204767,132.476413,9.822996,16.145175,0.0,NaN,NaN,NaN,NaN
min,30.138250,-85.936930,1.500000,3.580000,NaN,NaN,60.000000,50.000000,10.100000,18.100000,1.0,1.0,1.0,NaN,NaN
25%,30.203018,-85.834473,5.800000,6.600000,NaN,NaN,87.500000,82.500000,10.100000,20.725000,1.0,1.0,1.0,NaN,NaN
50%,30.241976,-85.780702,8.500000,7.270000,NaN,NaN,160.000000,120.000000,12.550000,27.500000,1.0,1.0,1.0,NaN,NaN
75%,30.324893,-85.643120,16.175000,10.512500,NaN,NaN,195.000000,277.500000,21.225000,29.100000,1.0,1.0,1.0,NaN,NaN
max,30.432762,-85.399180,40.200000,19.460000,NaN,NaN,360.000000,360.000000,38.000000,62.000000,1.0,1.0,1.0,NaN,NaN


### General Hurricane Data 
Hurricane data from 1900 to 2018

In [12]:
storm_data = pd.read_csv(r'data\storm_data_search_results (1).csv')
storm_data

EVENT_ID                         CZ_NAME_STR BEGIN_LOCATION  BEGIN_DATE  \
0     655883             COASTAL FRANKLIN (ZONE)                 09/01/2016   
1     655884              INLAND FRANKLIN (ZONE)                 09/01/2016   
2     655885              COASTAL WAKULLA (ZONE)                 09/01/2016   
3     655886               INLAND WAKULLA (ZONE)                 09/01/2016   
4     655887            COASTAL JEFFERSON (ZONE)                 09/01/2016   
5     655889             INLAND JEFFERSON (ZONE)                 09/01/2016   
6     657382               COASTAL TAYLOR (ZONE)                 09/01/2016   
7     657383                INLAND TAYLOR (ZONE)                 09/01/2016   
8     661501                 INDIAN RIVER (ZONE)                 10/06/2016   
9     661647                      BREVARD (ZONE)                 10/06/2016   
10    661648                      VOLUSIA (ZONE)                 10/06/2016   
11    722003                COASTAL PASCO (ZONE)                 09/08/2017   
12    722914              MAINLAND MONROE (ZONE)                 09/09/2017   
13    723287  FAR SOUTH MIAMI-DADE COUNTY (ZONE)                 09/09/2017   
14    723286            INLAND MIAMI-DADE (ZONE)                 09/09/2017   
15    723457           MONROE/MIDDLE KEYS (ZONE)                 09/09/2017   
16    723459            MONROE/UPPER KEYS (ZONE)                 09/09/2017   
17    723450            MONROE/LOWER KEYS (ZONE)                 09/09/2017   
18    722895       COASTAL COLLIER COUNTY (ZONE)                 09/09/2017   
19    723255        INLAND COLLIER COUNTY (ZONE)                 09/09/2017   
20    721738             INLAND CHARLOTTE (ZONE)                 09/10/2017   
21    721736                   INLAND LEE (ZONE)                 09/10/2017   
22    721768                      DE SOTO (ZONE)                 09/10/2017   
23    721739                    HIGHLANDS (ZONE)                 09/10/2017   
24    721737            COASTAL CHARLOTTE (ZONE)                 09/10/2017   
25    721735                  COASTAL LEE (ZONE)                 09/10/2017   
26    721803              INLAND SARASOTA (ZONE)                 09/10/2017   
27    721769                       HARDEE (ZONE)                 09/10/2017   
28    721791             COASTAL SARASOTA (ZONE)                 09/10/2017   
29    721815              COASTAL MANATEE (ZONE)                 09/10/2017   
30    721812               INLAND MANATEE (ZONE)                 09/10/2017   
31    721819                         POLK (ZONE)                 09/10/2017   
32    721999          INLAND HILLSBOROUGH (ZONE)                 09/10/2017   
33    721998         COASTAL HILLSBOROUGH (ZONE)                 09/10/2017   
34    722000                     PINELLAS (ZONE)                 09/10/2017   
35    790380                         LEON (ZONE)                 10/10/2018   
36    790381                      GADSDEN (ZONE)                 10/10/2018   
37    790382                      LIBERTY (ZONE)                 10/10/2018   
38    790383                      CALHOUN (ZONE)                 10/10/2018   
39    790384                      JACKSON (ZONE)                 10/10/2018   
40    790385                   WASHINGTON (ZONE)                 10/10/2018   
41    789475                  COASTAL BAY (ZONE)                 10/10/2018   
42    790373                   INLAND BAY (ZONE)                 10/10/2018   
43    790374                 COASTAL GULF (ZONE)                 10/10/2018   
44    790375                  INLAND GULF (ZONE)                 10/10/2018   
45    790376              INLAND FRANKLIN (ZONE)                 10/10/2018   
46    790377             COASTAL FRANKLIN (ZONE)                 10/10/2018   
47    790378               INLAND WAKULLA (ZONE)                 10/10/2018   
48    790379              COASTAL WAKULLA (ZONE)                 10/10/2018   

    BEGIN_TIME           EVENT_TYPE MAGNITUDE TOR_F_S

### NOAA API

In [13]:
#needed to make web requests
import requests
#store the data we get as a dataframe
import pandas as pd
#convert the response as a strcuctured json
import json
#mathematical operations on lists
import numpy as np
#parse the datetimes we get from NOAA
from datetime import datetime
#add the access token you got from NOAA
Token = 'mrMETAXWcTJKLduCSeekSAMcBlDEfGYZ'
#MIAMI LAKES 2.1 N, FL US
#https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:US1FLMD0006/detail
station_id = 'GHCND:US1FLFR0002'

In [14]:
#https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
#initialize lists to store data


#for each year from 2016-2019 ...
for year in range(2016, 2019):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&stationid=GHCND:US1FLFR0002&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)


working on year 2016
working on year 2017
working on year 2018


In [15]:
dates_temp = []
dates_prcp = []
temps = []
prcp = []
#get all items in the response which are average temperature readings
avg_temps = [item for item in d['results'] if item['datatype']=='AWND']
#get the date field from all average temperature readings
dates_temp += [item['date'] for item in avg_temps]
#get the actual average temperature from all average temperature readings
temps += [item['value'] for item in avg_temps]

KeyError: 'results'

In [ ]:
#https://towardsdatascience.com/getting-weather-data-in-3-easy-steps-8dc10cc5c859
#initialize lists to store data
dates_temp = []
dates_prcp = []
temps = []
prcp = []

#for each year from 2015-2019 ...
for year in range(2015, 2020):
    year = str(year)
    print('working on year '+year)
    
    #make the api call
    r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&stationid=GHCND:USW00023129&startdate='+year+'-01-01&enddate='+year+'-12-31', headers={'token':Token})
    #load the api response as a json
    d = json.loads(r.text)
    #get all items in the response which are average temperature readings
    avg_temps = [item for item in d['results'] if item['datatype']=='AWND']
    #get the date field from all average temperature readings
    dates_temp += [item['date'] for item in avg_temps]
    #get the actual average temperature from all average temperature readings
    temps += [item['value'] for item in avg_temps]

In [ ]:
#initialize dataframe
df_temp = pd.DataFrame()

#populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0*1.8 + 32 for v in temps]

In [ ]:
df_temp.describe()